# Scraping

In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Firefox

from src.TwitterScraper.scraper import get_tweet_data

#open the browser and logs into twitter
browser = Firefox()
browser.get('https://www.twitter.com/login')
sleep(2)
username = browser.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('orjanenes@gmail.com')
password = getpass()
getpasswordInput = browser.find_element_by_xpath('//input[@name="session[password]"]')
getpasswordInput.send_keys(password)
getpasswordInput.send_keys(Keys.RETURN)
sleep(2)

#Finds the search input and enters the search string
search_input = browser.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('#spacex')
search_input.send_keys(Keys.RETURN)

sleep(2)
#Clicks on the Latest tab
browser.find_element_by_link_text('Latest').click()
sleep(2)

#Get all the tweet data
data = []
tweet_ids = set()
last_position = browser.execute_script("return window.pageYOffset;")
scrolling = True

#scrape tweets until we dont find more tweets or our dataset has 1000 elements
while scrolling and len(data)<=1000:
    page_cards = browser.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)

    scroll_attempt = 0
    while True:
        #Check scroll pos
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = browser.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt +=1
            
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break
browser.close()

# Saving the data

In [ ]:
with open('spacex_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['Username', 'Handle', 'Timestamp', 'text', 'Emojis', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)